In [1]:
import os
import glob
import numpy as np
import cv2
from collections import defaultdict

In [2]:
def parse_dota_annotation(file_path):
    """Parses DOTA annotation file and returns a list of bounding boxes and labels."""
    annotations = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) < 9:  # Ensure there are enough elements
                continue
            coords = list(map(float, parts[:8]))  # First 8 are coordinates
            category = parts[8]  # Category name
            annotations.append((coords, category))
    return annotations

def load_annotations(directory):
    """Loads all DOTA annotations from a directory into a dictionary."""
    annotation_dict = {}
    for file_path in glob.glob(os.path.join(directory, "*.txt")):
        file_name = os.path.basename(file_path)
        annotation_dict[file_name] = parse_dota_annotation(file_path)
    return annotation_dict

def polygon_to_mask(polygon, shape):
    """Converts a polygon to a binary mask."""
    mask = np.zeros(shape, dtype=np.uint8)
    pts = np.array(polygon, dtype=np.int32).reshape(-1, 1, 2)
    cv2.fillPoly(mask, [pts], 1)
    return mask

def compute_iou(poly1, poly2, img_size=(1024, 1024)):
    """Computes the Intersection over Union (IoU) between two polygons."""
    mask1 = polygon_to_mask(poly1, img_size)
    mask2 = polygon_to_mask(poly2, img_size)
    
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    
    return intersection / union if union > 0 else 0

def evaluate_dota(val_dir, pred_dir, iou_threshold=0.5):
    """Evaluates predictions against ground truth annotations."""
    val_annotations = load_annotations(val_dir)
    pred_annotations = load_annotations(pred_dir)

    categories = set()
    for anns in val_annotations.values():
        for _, cat in anns:
            categories.add(cat)

    results = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})

    for file_name, gt_boxes in val_annotations.items():
        pred_boxes = pred_annotations.get(file_name, [])

        matched = set()
        for pred_poly, pred_cat in pred_boxes:
            best_iou = 0
            best_match = None

            for i, (gt_poly, gt_cat) in enumerate(gt_boxes):
                if gt_cat == pred_cat:
                    iou = compute_iou(pred_poly, gt_poly)
                    if iou > best_iou:
                        best_iou = iou
                        best_match = i

            if best_iou >= iou_threshold and best_match is not None:
                if best_match not in matched:
                    results[pred_cat]['TP'] += 1
                    matched.add(best_match)
                else:
                    results[pred_cat]['FP'] += 1
            else:
                results[pred_cat]['FP'] += 1

        for i, (_, gt_cat) in enumerate(gt_boxes):
            if i not in matched:
                results[gt_cat]['FN'] += 1

    # Compute Precision, Recall, and F1-score
    for cat, vals in results.items():
        TP, FP, FN = vals['TP'], vals['FP'], vals['FN']
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        results[cat]['Precision'] = precision
        results[cat]['Recall'] = recall
        results[cat]['F1-score'] = f1_score

    return results

In [3]:
# Example usage
val_dir = "data/brickkilns/test/annfiles"
pred_dir = "results/bihar/annfiles"

metrics = evaluate_dota(val_dir, pred_dir, iou_threshold=0.5)

# Print results
for category, values in metrics.items():
    print(f"Category: {category}")
    print(f"  TP: {values['TP']}, FP: {values['FP']}, FN: {values['FN']}")
    print(f"  Precision: {values['Precision']:.4f}, Recall: {values['Recall']:.4f}, F1-score: {values['F1-score']:.4f}")
    print("-" * 40)


Category: Zigzag
  TP: 830, FP: 712, FN: 170
  Precision: 0.5383, Recall: 0.8300, F1-score: 0.6530
----------------------------------------
Category: FCBK
  TP: 15, FP: 166, FN: 105
  Precision: 0.0829, Recall: 0.1250, F1-score: 0.0997
----------------------------------------
Category: CFCBK
  TP: 5, FP: 2, FN: 5
  Precision: 0.7143, Recall: 0.5000, F1-score: 0.5882
----------------------------------------


In [4]:
# Example usage
val_dir = "data/m0/val/annfiles"
pred_dir = "results/m0/val/annfiles"

metrics = evaluate_dota(val_dir, pred_dir, iou_threshold=0.5)

# Print results
for category, values in metrics.items():
    print(f"Category: {category}")
    print(f"  TP: {values['TP']}, FP: {values['FP']}, FN: {values['FN']}")
    print(f"  Precision: {values['Precision']:.4f}, Recall: {values['Recall']:.4f}, F1-score: {values['F1-score']:.4f}")
    print("-" * 40)


Category: Zigzag
  TP: 140, FP: 196, FN: 84
  Precision: 0.4167, Recall: 0.6250, F1-score: 0.5000
----------------------------------------
Category: FCBK
  TP: 4, FP: 13, FN: 78
  Precision: 0.2353, Recall: 0.0488, F1-score: 0.0808
----------------------------------------
Category: CFCBK
  TP: 2, FP: 1, FN: 15
  Precision: 0.6667, Recall: 0.1176, F1-score: 0.2000
----------------------------------------


In [5]:
# Example usage
val_dir = "data/m0/train/annfiles"
pred_dir = "results/m0/train/annfiles"

metrics = evaluate_dota(val_dir, pred_dir, iou_threshold=0.5)

# Print results
for category, values in metrics.items():
    print(f"Category: {category}")
    print(f"  TP: {values['TP']}, FP: {values['FP']}, FN: {values['FN']}")
    print(f"  Precision: {values['Precision']:.4f}, Recall: {values['Recall']:.4f}, F1-score: {values['F1-score']:.4f}")
    print("-" * 40)

Category: Zigzag
  TP: 582, FP: 730, FN: 328
  Precision: 0.4436, Recall: 0.6396, F1-score: 0.5239
----------------------------------------
Category: CFCBK
  TP: 10, FP: 2, FN: 42
  Precision: 0.8333, Recall: 0.1923, F1-score: 0.3125
----------------------------------------
Category: FCBK
  TP: 15, FP: 70, FN: 366
  Precision: 0.1765, Recall: 0.0394, F1-score: 0.0644
----------------------------------------
